### 멀티턴 대화 관리 방법

> 멀티턴(Multi-turn) 대화란?  
>  한 번의 질문-응답이 아니라, 여러 번의 대화(질문-답변-질문-답변...)가 이어지는 상황에서 **이전 대화 내용을 기억**하고, 그 맥락에 맞는 답변을 생성하는 기술

**4가지 방식의 멀티턴**

1. Message Passing 방식
2. ChatMessageHistory 객체 사용
3. RunnableWithMessageHistory 자동 관리
4. 대화 요약 및 히스토리 축약

> 참고: How to add memory to chatbots (<https://python.langchain.com/v0.2/docs/how_to/chatbots_memory/>)

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=gemini_api_key)

### 1. Message passing

**가장 기본적인 멀티턴 대화 관리 방법** 

- 대화 내용을 파이썬 **리스트**에 직접 저장하고, 매번 LLM에 전체 대화 기록을 전달
- **장점:** 구조가 단순하고, 동작 원리를 이해하기 쉬움
- **단점:** 대화가 길어질수록 리스트가 커지고, 관리가 번거로워짐
- **활용 예시:** 간단한 챗봇, 실험용 코드, 대화 기록이 짧은 경우

#### 1) 튜플 사용

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean.",
        ),
        ("placeholder", "{messages}"),
    ]
)

chain = prompt | llm

In [22]:
ai_msg = chain.invoke(
    {
        "messages": [
            (
                "human",
                "내 이름은 김일남이야. 나이는 99세야.",
            ),
            ("ai",  "그렇군요. 나이가 많으시네요!"),
            ("human", "내 나이는?"),
        ],
    }
)
print(ai_msg.content)

99세입니다.


In [7]:
history_list = []

while(True):
    user_input = input()

    if user_input == "종료": break
    
    history_list.append(
        (
            "human",
            user_input,
        )
    )
    
    print("CHAT_HISTORY:", history_list)
    
    ai_msg = chain.invoke(
        {
            "messages": history_list,
        }
    )
    
    print("AI:", ai_msg.content)

    history_list.append(
        (
            "ai",
            ai_msg.content,
        )
    )

CHAT_HISTORY: [('human', '내 이름은 김일남이야')]
AI: 김일남 님, 안녕하세요! 만나서 반갑습니다.
CHAT_HISTORY: [('human', '내 이름은 김일남이야'), ('ai', '김일남 님, 안녕하세요! 만나서 반갑습니다.'), ('human', '내 이름이 뭐야?')]
AI: 김일남 님이세요!


#### 2) 객체 사용

In [14]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage # LangChain에서 대화의 각 턴을 나타내는 데 사용되는 객체

history_list = [
    SystemMessage("You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean."),  # 챗봇의 역할을 정의
]

In [15]:
while(True):
    user_input = input()

    if user_input == "종료": break
    
    history_list.append(
        HumanMessage(user_input)
    )
    
    ai_msg = chain.invoke(
        {
            "messages": history_list,
        }
    )

    history_list.append(
        AIMessage(ai_msg.content)
    )
    
    print("CHAT_HISTORY:", history_list)
    
    print("AI:", ai_msg.content)


CHAT_HISTORY: [SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 김일남님. 만나서 반갑습니다!', additional_kwargs={}, response_metadata={})]
AI: 안녕하세요, 김일남님. 만나서 반갑습니다!
CHAT_HISTORY: [SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 김일남님. 만나서 반갑습니다!', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은?', additional_kwargs={}, response_metadata={}), AIMessage(content='김일남님이십니다.', additional_kwargs={}, response_metadata={})]
AI: 김일남님이십니다.


### 2. Chat history

LangChain의 **ChatMessageHistory** 객체를 활용한 멀티턴 대화 관리

- `add_user_message`, `add_ai_message` 등 메서드로 대화 기록을 쉽게 추가/관리 가능
- **장점:**  
  - 메시지 타입(user/ai)을 명확히 구분  
  - 대화 기록 관리가 편리
- **단점:** 여전히 직접 기록을 관리해야 하므로, 대화가 길어지면 불편할 수 있음
- **활용 예시:** 챗봇, 상담 서비스, 대화 기록이 중요한 서비스

In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message(
    "내 이름은 김일남이야. 나이는 99세야"
)

chat_history.add_ai_message("그렇군요. 나이가 많으시네요!")

chat_history.messages
# chat_history.clear()

[HumanMessage(content='내 이름은 김일남이야. 나이는 99세야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='그렇군요. 나이가 많으시네요!', additional_kwargs={}, response_metadata={})]

In [9]:
chat_history.add_user_message(
    "내 나이는?"
)

chat_history.messages

[HumanMessage(content='내 이름은 김일남이야. 나이는 99세야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='그렇군요. 나이가 많으시네요!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내 나이는?', additional_kwargs={}, response_metadata={})]

In [ ]:
# 답변 생성(response)
response = chain.invoke(
    {
        "messages": chat_history.messages,
    }
)

# 대화 히스토리에 답변(response) 저장
chat_history.add_ai_message(response)

print(chat_history.messages)

[HumanMessage(content='내 이름은 김일남이야. 나이는 99세야', additional_kwargs={}, response_metadata={}), AIMessage(content='그렇군요. 나이가 많으시네요!', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 나이는?', additional_kwargs={}, response_metadata={}), AIMessage(content='김일남 님께서는 99세라고 말씀해주셨습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--62d881c5-475f-4d4d-a1a0-aaae89acdee5-0', usage_metadata={'input_tokens': 64, 'output_tokens': 69, 'total_tokens': 133, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 52}})]


In [11]:
chat_history = ChatMessageHistory()

while(True):
    user_input = input()

    if user_input == "종료": break
    
    chat_history.add_user_message(user_input)
    
    response = chain.invoke(
        {
            "messages": chat_history.messages,
        }
    )
    
    chat_history.add_ai_message(response)

    print("chat_history.messages:", chat_history.messages)
    print("AI:", response.content)

chat_history.messages: [HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 김일남님. 만나서 반갑습니다!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--3e39a8e5-2085-4527-911f-4ef7663df693-0', usage_metadata={'input_tokens': 39, 'output_tokens': 36, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 25}})]
AI: 안녕하세요, 김일남님. 만나서 반갑습니다!
chat_history.messages: [HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 김일남님. 만나서 반갑습니다!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--3e39a8e5-2085-4527-911f-4ef7663df693-0', usage_metadata={'input_tokens': 39, 'o

### 3. Automatic history management

**RunnableWithMessageHistory** 활용 **세션별** 대화 **자동 기록** 관리

- 세션 ID만 지정, 각 사용자의 대화 기록을 별도로 저장/관리
- **장점:**
  - 실제 서비스(여러 사용자 동시 접속)에서 매우 유용  
  - 코드가 간결해지고, 대화 기록 관리가 자동화됨
- **단점:** 구조를 이해하려면 약간의 학습이 필요
- **활용 예시:** 실시간 상담 챗봇, 멀티유저 게임, 고객센터 등

In [27]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = prompt | llm

In [28]:
from langchain_core.runnables.history import RunnableWithMessageHistory

# 세션별 채팅 히스토리 관리
chat_histories = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    if session_id not in chat_histories:
        chat_histories[session_id] = ChatMessageHistory()
    return chat_histories[session_id]

chain_with_message_history = RunnableWithMessageHistory(
    chain, # 실행할 Runnable 객체
    get_session_history, # 세션 ID에 따라 대화 기록을 가져오는 함수
    input_messages_key="input", # 입력 메시지의 Key
    history_messages_key="chat_history", # 대화 히스토리 메시지의 Key
)

In [29]:
config = {"configurable": {"session_id": "kim1"}}  # 세션 ID 설정

chain_with_message_history.invoke(
    {"input": "내 이름은 김일남이야. 나이는 99세야."},
    config=config, # 세션 ID == kim1
)

AIMessage(content='안녕하세요, 김일남 님! 99세이시군요. 만나뵙게 되어 반갑습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--b3b47d65-6c48-474d-881a-91a7b3263c66-0', usage_metadata={'input_tokens': 39, 'output_tokens': 52, 'total_tokens': 91, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 30}})

In [30]:
chain_with_message_history.invoke(
    {"input": "내 나이는?"}, {"configurable": {"session_id": "kim1"}}
)

AIMessage(content='김일남 님께서는 99세이십니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--73e3d1a8-4fea-4b55-8422-417a6ffeda19-0', usage_metadata={'input_tokens': 67, 'output_tokens': 43, 'total_tokens': 110, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 28}})

In [31]:
chain_with_message_history.invoke(
    {"input": "내 나이는?"}, {"configurable": {"session_id": "kim2"}} # kim2 세션의 경우
)

AIMessage(content='저는 귀하의 개인 정보를 알 수 없기 때문에 나이를 알려드릴 수 없습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--23075873-4e14-4fa3-92a6-3964aca2d241-0', usage_metadata={'input_tokens': 27, 'output_tokens': 70, 'total_tokens': 97, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 52}})

In [ ]:
for r in chain_with_message_history.stream(
    {"input": "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐"},
    config=config,
):
    print(r.content, end="", flush=True)

김일남 님의 성함인 "김일남"은 한국에서 매우 흔하고 전통적인 이름입니다. 특히 "김"은 한국에서 가장 많은 성씨 중 하나이므로, **한국분이실 가능성이 높다고 생각합니다.**

만약 한국분이시라면, 한국 문화에 대해 몇 가지 말씀드릴 수 있습니다.

한국 문화는 오랜 역사와 전통을 바탕으로 발전해 왔으며, 다음과 같은 특징들을 가지고 있습니다:

1.  **공동체 의식과 예의:** 가족과 공동체를 중요하게 생각하며, 연장자나 지위에 따른 존경과 예의를 매우 중시합니다. 호칭 사용이나 인사 방식(예: 고개 숙여 인사하기)에서 이러한 면모를 엿볼 수 있습니다.
2.  **역동성과 현대성:** 전통을 보존하면서도 빠르게 변화하고 발전하는 역동적인 문화를 가지고 있습니다. K-팝, K-드라마, 영화 등 한류 콘텐츠는 전 세계적으로 큰 인기를 얻고 있습니다.
3.  **음식 문화:** 김치, 비빔밥, 불고기 등 다채롭고 건강한 음식 문화를 자랑합니다. 밥과 국, 다양한 반찬을 함께 먹는 것이 일반적이며, 함께 음식을 나누는 것을 중요하게 여깁니다.
4.  **교육열과 근면성:** 교육에 대한 열의가 매우 높고, 근면하고 성실하게 일하는 것을 미덕으로 여깁니다. 이러한 가치관은 한국의 빠른 경제 성장에 크게 기여했습니다.
5.  **전통 예술과 명절:** 판소리, 한국무용 등 전통 예술을 보존하고 있으며, 설날(음력 1월 1일)과 추석(음력 8월 15일) 같은 명절에는 온 가족이 모여 차례를 지내고 전통 음식을 나누며 조상과 가족의 의미를 되새깁니다.

김일남 님께서 한국분이시라면, 99세라는 연세에 걸맞게 이러한 한국의 전통과 변화를 모두 경험하며 살아오셨을 것이라 생각하니 더욱 존경스럽습니다.

### 4. Modifying chat history

대화가 길어질 때, **이전 대화를 요약해서 히스토리를 압축**하는 방법입니다.

- LLM의 입력 토큰 한계 극복 가능
- 요약된 메시지로 히스토리 대체, 중요한 정보만 남기고 불필요한 대화는 줄임
- **장점:**  
  - 긴 대화도 효율적으로 관리  
  - LLM의 맥락 유지 능력 향상
- **활용 예시:** 장시간 상담, 장기 프로젝트 관리, 대화 요약 기능 제공 서비스

In [31]:
chat_history = ChatMessageHistory()

chat_history.add_user_message("내 이름은 김일남이야.")
chat_history.add_ai_message("안녕하세요, 김일남님! 무엇을 도와드릴까요?")
chat_history.add_user_message("날씨 좋은 날 들을만 한 노래 추천해주세요.")
chat_history.add_ai_message("볼빨간사춘기 – 여행을 추천해요.")

chat_history.messages
# chat_history.clear()

[HumanMessage(content='내 이름은 김일남이야.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 김일남님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='날씨 좋은 날 들을만 한 노래 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='볼빨간사춘기 – 여행을 추천해요.', additional_kwargs={}, response_metadata={})]

In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. You must answer in Korean.",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | llm

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history, # 단일 사용자 환경
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [33]:
chain_with_message_history.invoke(
    {"input": "내 이름은 뭐야?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='고객님의 이름은 김일남입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--1cc8fca2-5087-4324-ab63-0ff90986772f-0', usage_metadata={'input_tokens': 77, 'output_tokens': 65, 'total_tokens': 142, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 55}})

In [34]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = chat_history.messages

    if len(stored_messages) == 0:
        return False
    
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "Distill the above chat messages into a single summary message. Include as many specific details as you can. Please, use Korean",
            ),
        ]
    )
    
    summarization_chain = summarization_prompt | llm

    # chat_history 에 저장된 대화 기록을 요약프롬프트에 입력 & 결과 저장
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    print("summary_message: ", summary_message)
    
    # chat_history 에 저장되어있던 기록 지우기
    chat_history.clear()

    # 생성된 새로운 요약내용으로 기록 채우기
    chat_history.add_message(summary_message)

    return True

chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [35]:
chain_with_summarization.invoke(
    {"input": "내 이름은?"},
    {"configurable": {"session_id": "unused"}},
)

summary_message:  content="사용자 김일남님은 **날씨 좋은 날 들을 만한 노래**를 추천해달라고 요청하셨고, 저는 **볼빨간사춘기의 '여행'**을 추천해드렸습니다. 이후 김일남님은 **자신의 이름을 다시 확인하고자 하셨고**, 저는 **'김일남'이라고 정확히** 알려드렸습니다." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--b14af18e-e57b-4ab9-a68d-3b93bd8f080a-0' usage_metadata={'input_tokens': 92, 'output_tokens': 1099, 'total_tokens': 1191, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1021}}


AIMessage(content='사용자님의 이름은 **김일남**님이십니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--19078fca-7e2f-41f0-be9b-e3e91af5d6a7-0', usage_metadata={'input_tokens': 106, 'output_tokens': 64, 'total_tokens': 170, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 50}})

In [36]:
chain_with_summarization.invoke(
    {"input": "그 가수는 남자인가요 여자인가요?"},
    {"configurable": {"session_id": "unused"}},
)

summary_message:  content="사용자 김일남님은 날씨 좋은 날 들을 만한 노래를 추천해달라고 요청하셨고, 이에 저는 볼빨간사춘기의 '여행'을 추천해드렸습니다. 이후 김일남님은 자신의 이름을 확인하고자 '내 이름은?'이라고 질문하셨고, 저는 '김일남'님이라고 정확히 알려드렸습니다." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--df492e30-90d5-4e73-97cd-6352393df3ed-0' usage_metadata={'input_tokens': 125, 'output_tokens': 969, 'total_tokens': 1094, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 891}}


AIMessage(content='그 가수는 여성입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--5b9f6dea-df28-47d8-bd2e-4a574aaa59e2-0', usage_metadata={'input_tokens': 114, 'output_tokens': 171, 'total_tokens': 285, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 165}})